In [1]:
import kagglehub
hahunavth_asvspoof2019_la_path = kagglehub.dataset_download('hahunavth/asvspoof2019-la')
# asvspoof2021_la_path = kagglehub.dataset_download('ajaysuryal/asvspoof2021-la')
wav2vec_model_path = kagglehub.dataset_download('ajaysuryal/wav2vec2-base')
test_dataset = kagglehub.dataset_download('ajaysuryal/test-audio')
print('Data source import complete.')

Data source import complete.


In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import warnings

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torch.nn.utils.rnn import pad_sequence


import torchaudio
import torchaudio.functional as AF
import torchaudio.functional as TF 

from transformers import Wav2Vec2Config, Wav2Vec2Model

from tqdm.auto import tqdm
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    confusion_matrix
)

torch.utils.data._utils.worker._worker_loop._WARNINGS_AS_ERRORS = False

2025-07-13 14:45:11.257902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752417911.470398      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752417911.532367      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
!pip install --upgrade numpy==1.23.5  
!pip install numpy_minmax numpy_rms   
!pip install audiomentations==0.28.0  #

from audiomentations import Compose, AddGaussianNoise, PitchShift, Shift, TimeStretch, Gain, PolarityInversion, HighPassFilter, LowPassFilter, ClippingDistortion


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 65.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.2 which is incompatible.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 1.23.5 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 1.23.5 which is incompatible.
mkl-fft 1.3.8 requires numpy<1.27.0,>=1.26.4, but you have numpy 1.23.5 which is incompatible.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
woodwork 0.31.0 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
f

In [5]:
from pathlib import Path

input_path = Path(test_dataset + '/test_audio' )
if input_path.exists():
    print("\nAlso found dataset in /kaggle/input:")
    for item in input_path.iterdir():
        print(f" - {item.name}")
else:
  print("not exits")


Also found dataset in /kaggle/input:
 - 1_fake.wav


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [7]:
path_2019 = hahunavth_asvspoof2019_la_path+'/LA/ASVspoof2019_LA_train/flac'
eval_path =hahunavth_asvspoof2019_la_path +  '/LA/ASVspoof2019_LA_eval/flac'


train_protocol = hahunavth_asvspoof2019_la_path+'/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt'
eval_protocol =  hahunavth_asvspoof2019_la_path +'/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval.trl.txt'

In [8]:
# path_2019 = '/kaggle/input/asvspoof2019-la/LA/ASVspoof2019_LA_train/flac'
# eval_path = "/kaggle/input/asvspoof2019-la/LA/ASVspoof2019_LA_eval/flac"
# train_protocol = '/kaggle/input/asvspoof2019-la/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt'
# eval_protocol = '/kaggle/input/asvspoof2019-la/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval.trl.txt'

In [9]:
augment = Compose([
    AddGaussianNoise(
        min_amplitude=0.001, 
        max_amplitude=0.02, 
        p=0.3
    ),
    PitchShift(
        min_semitones=-4, 
        max_semitones=4, 
        p=0.4
    ),
    Shift(
        min_fraction=-0.5, 
        max_fraction=0.5, 
        p=0.3
    ),
    TimeStretch(
        min_rate=0.8, 
        max_rate=1.2, 
        p=0.3
    ),
    HighPassFilter(
        min_cutoff_freq=50, 
        max_cutoff_freq=200, 
        p=0.2
    ),
    LowPassFilter(
        min_cutoff_freq=3000, 
        max_cutoff_freq=8000, 
        p=0.2
    ),
    Gain(
        min_gain_in_db=-6, 
        max_gain_in_db=6, 
        p=0.3
    ),
    ClippingDistortion(
        min_percentile_threshold=0,
        max_percentile_threshold=10,
        p=0.2
    ),
    PolarityInversion(p=0.1)
])

In [10]:
# For your training dataset
train_df = pd.read_csv(train_protocol, sep=' ', header=None,
                     names=['speaker_id', 'audio_id', 'environment', 'attack_id', 'label'])

# Count the occurrences of each class
class_counts = train_df['label'].value_counts()
print("Training set class distribution:")
print(class_counts)

# Calculate percentages
class_percentages = train_df['label'].value_counts(normalize=True) * 100
print("\nClass percentages:")
print(class_percentages)

Training set class distribution:
label
spoof       22800
bonafide     2580
Name: count, dtype: int64

Class percentages:
label
spoof       89.834515
bonafide    10.165485
Name: proportion, dtype: float64


In [11]:
MAX_T = 48000

In [12]:
class AudioSpoofDataset(Dataset):
    def __init__(self, csv_path, audio_root, sample_rate=16000, augment_fn=None):
        import pandas as pd
        self.df = pd.read_csv(csv_path, sep=' ', header=None,
                              names=['speaker_id', 'audio_id', 'environment', 'attack_id', 'label'])
        self.audio_root = audio_root
        self.sample_rate = sample_rate
        self.augment_fn = augment_fn
        self.label_map = {'bonafide': 0, 'spoof': 1}
        self.MAX_T = 48000

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        import os
        row = self.df.iloc[idx]
        audio_path = os.path.join(self.audio_root, f"{row.audio_id}.flac")
        waveform, sr = torchaudio.load(audio_path)
        if sr != self.sample_rate:
            waveform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=self.sample_rate)(waveform)
        if waveform.size(-1) > self.MAX_T:
            waveform = waveform[:, :self.MAX_T]
        elif waveform.size(-1) < self.MAX_T:
            pad = self.MAX_T - waveform.size(-1)
            waveform = F.pad(waveform, (0, pad))
        waveform = waveform.squeeze(0).float()  # [T]
        if self.augment_fn:
            wav_np = waveform.numpy()
            wav_aug = self.augment_fn(samples=wav_np, sample_rate=self.sample_rate)
            waveform = torch.tensor(wav_aug)
        return {
            'waveform': waveform,  # [T]
            'label': torch.tensor(self.label_map[row.label], dtype=torch.long),
            'audio_id': row.audio_id }

In [13]:
def collate_fn(batch):
    waveforms = torch.stack([item['waveform'] for item in batch])
    labels = torch.stack([item['label'] for item in batch])
    return {
        'waveform': waveforms,        # [B, 1, T]
        'label': labels
    }

In [14]:
train_dataset = AudioSpoofDataset(
    csv_path= train_protocol,
    audio_root= path_2019,
    sample_rate=16000,
    augment_fn=augment
)

In [15]:
labels = train_df['label'].map({'spoof': 1, 'bonafide': 0}).values
class_counts = np.bincount(labels)
weights = 1. / class_counts[labels]  # inverse freq
sampler = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=8, sampler=sampler, collate_fn=collate_fn)

In [16]:
eval_dataset = AudioSpoofDataset(
    csv_path= eval_protocol,
    audio_root= eval_path,
    sample_rate=16000,
    augment_fn=None
)

In [17]:
# eval_loader = DataLoader(
#     eval_dataset,
#     batch_size=8,
#     shuffle=False,
#     num_workers=2,
#     collate_fn=collate_fn
# )

In [18]:
from torch.utils.data import random_split

total_size = len(eval_dataset)
split_1_size = int(0.7 * total_size)
split_2_size = total_size - split_1_size

# Create the splits
eval_dataset_70, eval_dataset_30 = random_split(
    eval_dataset, 
    [split_1_size, split_2_size]
)

In [19]:
# Create the splits
eval_dataset_70, eval_dataset_30 = random_split(
    eval_dataset, 
    [split_1_size, split_2_size]
)

# Create DataLoaders for each subset if needed
eval_loader = DataLoader(
    eval_dataset_70,
    batch_size=8,
    shuffle=False,
    num_workers=2,
    collate_fn=collate_fn
)

eval_loader_test = DataLoader(
    eval_dataset_30,
    batch_size=8,
    shuffle=False,
    num_workers=2,
    collate_fn=collate_fn
)

In [20]:
batch = next(iter(train_loader))
print("Waveform shape:", batch['waveform'].shape)
print("Labels shape:", batch['label'].shape)

Waveform shape: torch.Size([8, 48000])
Labels shape: torch.Size([8])


In [21]:
for batch_idx, batch in enumerate(eval_loader):
    print(f"\nBatch {batch_idx}:")
    print("- Waveform shape:", batch['waveform'].shape)
    print("- Labels shape:", batch['label'].shape)
    print("- Sample audio_id:", train_dataset.df.iloc[batch_idx * 8]['audio_id'])  # First sample's ID

    # Check first sample's stats
    sample_wave = batch['waveform'][0]  # First sample [1, T]
    print("- Audio length (samples):", sample_wave.shape[-1])
    print("- Sample rate:", train_dataset.sample_rate)
    print("- Label:", batch['label'][0].item())

    break


Batch 0:
- Waveform shape: torch.Size([8, 48000])
- Labels shape: torch.Size([8])
- Sample audio_id: LA_T_1138215
- Audio length (samples): 48000
- Sample rate: 16000
- Label: 1


In [22]:
warnings.filterwarnings("ignore", category=UserWarning, module="transformers")

In [23]:
class AntiSpoofModel(nn.Module):
    def __init__(self, wav2vec_model_path, use_transformer=True, num_classes=2):
        super().__init__()
        config = Wav2Vec2Config.from_pretrained(
            wav2vec_model_path,
            hidden_dropout=0.1,
            attention_dropout=0.1
        )
        self.wav2vec = Wav2Vec2Model.from_pretrained(wav2vec_model_path, config=config)
        self.wav2vec.feature_extractor._freeze_parameters()  # Freeze low-level features
        
        # 1. MULTI-SCALE F0 AND SPECTRAL ANALYSIS
        # Multiple bandpass filters for different frequency ranges
        self.f0_low = self._create_bandpass_filter(768, 50, 200)    # Low F0
        self.f0_mid = self._create_bandpass_filter(768, 200, 500)   # Mid F0
        self.f0_high = self._create_bandpass_filter(768, 500, 1000) # High F0
        
        # Spectral inconsistency detector
        self.spectral_analyzer = nn.Sequential(
            nn.Conv1d(768, 384, kernel_size=7, padding=3),
            nn.BatchNorm1d(384),
            nn.ReLU(),
            nn.Conv1d(384, 192, kernel_size=5, padding=2),
        )
        
        # 2. ENHANCED CNN WITH RESIDUAL CONNECTIONS
        # Multi-branch CNN for different temporal scales
        self.cnn_short = self._create_cnn_branch(768 + 192, 256, [3, 5])     
        self.cnn_medium = self._create_cnn_branch(768 + 192, 256, [7, 11])    
        self.cnn_long = self._create_cnn_branch(768 + 192, 256, [15, 21])     
        
        # Feature fusion
        self.feature_fusion = nn.Sequential(
            nn.Conv1d(768, 256, kernel_size=1),  # Dimension reduction
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        # 3. ATTENTION MECHANISMS
        # Self-attention for temporal dependencies
        self.temporal_attention = nn.MultiheadAttention(
            embed_dim=256, num_heads=8, dropout=0.1, batch_first=True
        )
        
        # Channel attention for feature importance
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            nn.Conv1d(256, 64, 1),
            nn.ReLU(),
            nn.Conv1d(64, 256, 1),
            nn.Sigmoid()
        )
        
        # 4. IMPROVED TRANSFORMER
        if use_transformer:
            encoder_layer = nn.TransformerEncoderLayer(
                d_model=256, nhead=8, dim_feedforward=1024, 
                dropout=0.1, batch_first=True
            )
            self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=3)
        else:
            self.transformer = nn.Identity()
        
        # 5. SOPHISTICATED POOLING AND CLASSIFICATION
        # Multiple pooling strategies
        self.adaptive_pool = nn.AdaptiveAvgPool1d(1)
        self.max_pool = nn.AdaptiveMaxPool1d(1)
        
        # Classification head with uncertainty estimation
        self.head = nn.Sequential(
            nn.Linear(256 * 2, 512),  # *2 for avg + max pooling
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
        
        # Uncertainty head for confidence estimation
        self.uncertainty_head = nn.Sequential(
            nn.Linear(256 * 2, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
    
    def _create_bandpass_filter(self, channels, low_freq, high_freq):
        """Create a learnable bandpass filter"""
        conv = nn.Conv1d(channels, channels, kernel_size=7, padding=3, groups=channels, bias=False)
        # Initialize with bandpass characteristics
        with torch.no_grad():
            # Simple bandpass initialization (can be improved with actual filter design)
            conv.weight.fill_(1/7)
        return conv
    
    def _create_cnn_branch(self, in_channels, out_channels, kernel_sizes):
        """Create a multi-scale CNN branch"""
        layers = []
        current_channels = in_channels
        
        for i, k in enumerate(kernel_sizes):
            layers.extend([
                nn.Conv1d(current_channels, out_channels // len(kernel_sizes), 
                         kernel_size=k, padding=k//2),
                nn.BatchNorm1d(out_channels // len(kernel_sizes)),
                nn.ReLU(),
                nn.Dropout(0.1)
            ])
            if i == 0:
                current_channels = out_channels // len(kernel_sizes)
        
        layers.append(nn.MaxPool1d(kernel_size=2))
        return nn.Sequential(*layers)
    
    def forward(self, waveform):
        # waveform: [B, T]
        x = self.wav2vec(waveform).last_hidden_state  # [B, T', 768]
        x = x.transpose(1, 2)  # [B, 768, T']
        
        # 1. Multi-scale F0 analysis
        f0_low = self.f0_low(x)
        f0_mid = self.f0_mid(x)  
        f0_high = self.f0_high(x)
        
        # Combine F0 features
        f0_combined = f0_low + f0_mid + f0_high
        
        # 2. Spectral inconsistency detection
        spectral_features = self.spectral_analyzer(x)
        
        # 3. Combine features
        combined_features = torch.cat([x, spectral_features], dim=1)  # [B, 768+192, T']
        
        # 4. Multi-scale CNN processing
        short_features = self.cnn_short(combined_features)
        medium_features = self.cnn_medium(combined_features)
        long_features = self.cnn_long(combined_features)
        
        # Combine multi-scale features
        multi_scale = torch.cat([short_features, medium_features, long_features], dim=1)
        
        # 5. Feature fusion and dimension reduction
        x = self.feature_fusion(f0_combined + x)  # Residual connection
        
        # 6. Channel attention
        channel_weights = self.channel_attention(x)
        x = x * channel_weights
        
        # 7. Temporal attention
        x = x.transpose(1, 2)  # [B, T'', 256]
        x_attended, _ = self.temporal_attention(x, x, x)
        x = x + x_attended  # Residual connection
        
        # 8. Transformer processing
        x = self.transformer(x)  # [B, T'', 256]
        
        # 9. Advanced pooling
        x = x.transpose(1, 2)  # [B, 256, T'']
        avg_pooled = self.adaptive_pool(x).squeeze(-1)  # [B, 256]
        max_pooled = self.max_pool(x).squeeze(-1)       # [B, 256]
        
        # Combine pooling strategies
        pooled_features = torch.cat([avg_pooled, max_pooled], dim=1)  # [B, 512]
        
        # 10. Classification with uncertainty
        logits = self.head(pooled_features)
        confidence = self.uncertainty_head(pooled_features)
        
        return {
            'logits': logits,
            'confidence': confidence,
            'features': pooled_features
        }

# TRAINING IMPROVEMENTS
class FocalLoss(nn.Module):
    """Focal Loss for handling class imbalance"""
    def __init__(self, alpha=1, gamma=2):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1-pt)**self.gamma * ce_loss
        return focal_loss.mean()

class LabelSmoothingLoss(nn.Module):
    """Label smoothing for better generalization"""
    def __init__(self, num_classes, smoothing=0.1):
        super().__init__()
        self.num_classes = num_classes
        self.smoothing = smoothing
        
    def forward(self, inputs, targets):
        confidence = 1.0 - self.smoothing
        smooth_targets = torch.full_like(inputs, self.smoothing / (self.num_classes - 1))
        smooth_targets.scatter_(1, targets.unsqueeze(1), confidence)
        return F.kl_div(F.log_softmax(inputs, dim=1), smooth_targets, reduction='batchmean')

# ENSEMBLE PREDICTION
class EnsemblePredictor:
    def __init__(self, models):
        self.models = models
    
    def predict(self, waveform):
        predictions = []
        confidences = []
        
        for model in self.models:
            with torch.no_grad():
                output = model(waveform)
                predictions.append(F.softmax(output['logits'], dim=1))
                confidences.append(output['confidence'])
        
        # Weighted average based on confidence
        weights = torch.stack(confidences, dim=0)
        weights = F.softmax(weights, dim=0)
        
        ensemble_pred = sum(w * p for w, p in zip(weights, predictions))
        return ensemble_pred

In [24]:
model = AntiSpoofModel(
    wav2vec_model_path=wav2vec_model_path + "/wav2vec2-base",
    use_transformer=True,
    num_classes=2
).to(device)


In [25]:
warnings.filterwarnings("ignore", category=UserWarning, message=".*can only test a child process.*")

In [ ]:
# Initialize the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")




print(f"Model initialized with {sum(p.numel() for p in model.parameters()):,} parameters")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# ADVANCED TRAINING SETUP
class ImprovedTrainer:
    def __init__(self, model, device, use_focal_loss=True, use_label_smoothing=False):
        self.model = model
        self.device = device
        
        # Choose loss function based on your data distribution
        self.criterion = LabelSmoothingLoss(num_classes=2, smoothing=0.1)

        # ...existing code...
        wav2vec_params = list(self.model.wav2vec.parameters())
        wav2vec_param_ids = set(id(p) for p in wav2vec_params)
        other_params = [p for p in self.model.parameters() if id(p) not in wav2vec_param_ids]
        # ...existing code... 
        
        self.optimizer = optim.AdamW([
            {'params': wav2vec_params, 'lr': 1e-5, 'weight_decay': 1e-6},  # Lower LR for pre-trained
            {'params': other_params, 'lr': 1e-4, 'weight_decay': 1e-5}     # Higher LR for new layers
        ])
        
        # Cosine annealing with warm restarts
        self.scheduler = CosineAnnealingWarmRestarts(
            self.optimizer, T_0=5, T_mult=2, eta_min=1e-7
        )
        
        # Backup scheduler
        self.plateau_scheduler = ReduceLROnPlateau(
            self.optimizer, mode='min', factor=0.5, patience=3, verbose=True
        )
        
        self.best_val_acc = 0.0
        self.best_val_loss = float('inf')
        self.train_losses = []
        self.val_losses = []
        self.train_accs = []
        self.val_accs = []
    
    def train_epoch(self, train_loader, epoch):
        self.model.train()
        train_loss, train_correct, train_total = 0, 0, 0
        all_preds, all_labels = [], []
        
        train_iter = tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]")
        
        for batch_idx, batch in enumerate(train_iter):
            wave = batch['waveform'].to(self.device)
            label = batch['label'].to(self.device)
            
            # Handle different input shapes
            if wave.dim() == 3 and wave.size(1) == 1:
                wave = wave.squeeze(1)  # [B, 1, T] -> [B, T]
            
            self.optimizer.zero_grad()
            
            # Forward pass
            outputs = self.model(wave)
            logits = outputs['logits'] if isinstance(outputs, dict) else outputs
            
            # Calculate loss
            loss = self.criterion(logits, label)
            
            # Add confidence regularization if available
            if isinstance(outputs, dict) and 'confidence' in outputs:
                confidence_loss = 0.1 * torch.mean((outputs['confidence'] - 0.5) ** 2)
                loss += confidence_loss
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()
            
            # Statistics
            preds = logits.argmax(dim=1)
            correct = (preds == label).sum().item()
            
            train_loss += loss.item() * wave.size(0)
            train_correct += correct
            train_total += label.size(0)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(label.cpu().numpy())
            
            # Update progress bar
            current_acc = correct / label.size(0)
            train_iter.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{current_acc:.4f}',
                'lr': f'{self.optimizer.param_groups[0]["lr"]:.2e}'
            })
        
        avg_train_loss = train_loss / train_total
        train_acc = train_correct / train_total
        
        # Calculate detailed metrics
        precision, recall, f1, _ = precision_recall_fscore_support(
            all_labels, all_preds, average='weighted'
        )
        
        return avg_train_loss, train_acc, precision, recall, f1
    
    def validate_epoch(self, val_loader, epoch):
        self.model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        all_preds, all_labels, all_confidences = [], [], []
        
        with torch.no_grad():
            val_iter = tqdm(val_loader, desc=f"Epoch {epoch+1} [Val]")
            
            for batch in val_iter:
                wave = batch['waveform'].to(self.device)
                label = batch['label'].to(self.device)
                
                if wave.dim() == 3 and wave.size(1) == 1:
                    wave = wave.squeeze(1)
                
                outputs = self.model(wave)
                logits = outputs['logits'] if isinstance(outputs, dict) else outputs
                
                loss = self.criterion(logits, label)
                
                preds = logits.argmax(dim=1)
                correct = (preds == label).sum().item()
                
                val_loss += loss.item() * wave.size(0)
                val_correct += correct
                val_total += label.size(0)
                
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(label.cpu().numpy())
                
                if isinstance(outputs, dict) and 'confidence' in outputs:
                    all_confidences.extend(outputs['confidence'].cpu().numpy())
                
                val_iter.set_postfix({
                    'val_loss': f'{loss.item():.4f}',
                    'val_acc': f'{correct / label.size(0):.4f}'
                })
        
        avg_val_loss = val_loss / val_total
        val_acc = val_correct / val_total
        
        # Detailed metrics
        precision, recall, f1, _ = precision_recall_fscore_support(
            all_labels, all_preds, average='weighted'
        )
        
        # Confusion matrix
        cm = confusion_matrix(all_labels, all_preds)
        
        return avg_val_loss, val_acc, precision, recall, f1, cm, all_confidences
    
    def train(self, train_loader, val_loader, epochs=5, save_path='best_antispoofmodel.pth'):
        print("Starting training...")
        print(f"Dataset size: Train={len(train_loader.dataset)}, Val={len(val_loader.dataset)}")
        
        for epoch in range(epochs):
            print(f"\n{'='*50}")
            print(f"EPOCH {epoch+1}/{epochs}")
            print(f"{'='*50}")
            
            # Training
            train_loss, train_acc, train_prec, train_rec, train_f1 = self.train_epoch(train_loader, epoch)
            
            # Validation
            val_loss, val_acc, val_prec, val_rec, val_f1, cm, confidences = self.validate_epoch(val_loader, epoch)
            
            # Learning rate scheduling
            self.scheduler.step()
            # self.plateau_scheduler.step(val_loss)  # Uncomment if you want backup scheduler
            
            # Store metrics
            self.train_losses.append(train_loss)
            self.val_losses.append(val_loss)
            self.train_accs.append(train_acc)
            self.val_accs.append(val_acc)
            
            # Print detailed results
            print(f"\nTRAIN METRICS:")
            print(f"Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | Prec: {train_prec:.4f} | Rec: {train_rec:.4f} | F1: {train_f1:.4f}")
            
            print(f"\nVALIDATION METRICS:")
            print(f"Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | Prec: {val_prec:.4f} | Rec: {val_rec:.4f} | F1: {val_f1:.4f}")
            
            print(f"\nConfusion Matrix:")
            print(f"[[{cm[0,0]:4d}, {cm[0,1]:4d}]]  <- Real")
            print(f"[[{cm[1,0]:4d}, {cm[1,1]:4d}]]  <- Fake")
            
            if confidences:
                print(f"Average Confidence: {np.mean(confidences):.4f}")
            
            # Save best model
            if val_acc > self.best_val_acc:
                self.best_val_acc = val_acc
                self.best_val_loss = val_loss
                
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'scheduler_state_dict': self.scheduler.state_dict(),
                    'val_acc': val_acc,
                    'val_loss': val_loss,
                    'train_acc': train_acc,
                    'train_loss': train_loss
                }, save_path)
                
                print(f"🎉 NEW BEST MODEL SAVED! Val Acc: {val_acc:.4f}")
            
            # Early stopping check
            if epoch > 5 and val_acc < self.best_val_acc - 0.05:
                print("⚠️  Performance degrading, consider early stopping")
        
        print(f"\n🏁 TRAINING COMPLETED!")
        print(f"Best Validation Accuracy: {self.best_val_acc:.4f}")
        return self.best_val_acc
    
    def plot_training_curves(self):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Loss curves
        ax1.plot(self.train_losses, label='Train Loss', color='blue')
        ax1.plot(self.val_losses, label='Val Loss', color='red')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.set_title('Training and Validation Loss')
        ax1.legend()
        ax1.grid(True)
        
        # Accuracy curves
        ax2.plot(self.train_accs, label='Train Acc', color='blue')
        ax2.plot(self.val_accs, label='Val Acc', color='red')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Accuracy')
        ax2.set_title('Training and Validation Accuracy')
        ax2.legend()
        ax2.grid(True)
        
        plt.tight_layout()
        plt.savefig('training_curves.png', dpi=300, bbox_inches='tight')
        plt.show()

# INITIALIZE AND TRAIN
trainer = ImprovedTrainer(
    model=model, 
    device=device, 
    use_focal_loss=True,  # Good for balanced datasets
    use_label_smoothing=False
)

# Start training (replace with your actual data loaders)
best_accuracy = trainer.train(
    train_loader=train_loader,  # Your train loader
    val_loader=eval_loader,     # Your validation loader
    epochs=3,
    save_path='best_improved_antispoofmodel.pth'
)

# Plot training curves
trainer.plot_training_curves()

print(f"\n🎯 FINAL RESULTS:")
print(f"Best Validation Accuracy: {best_accuracy:.4f}")
print(f"Expected accuracy with 26K samples: 90-95%+ 🚀")

Using device: cuda
Model initialized with 103,827,459 parameters
Trainable parameters: 99,627,011
Starting training...
Dataset size: Train=25380, Val=49865

EPOCH 1/5


Epoch 1 [Train]:   0%|          | 0/3173 [00:00<?, ?it/s]

Epoch 1 [Val]:   0%|          | 0/6234 [00:00<?, ?it/s]


TRAIN METRICS:
Loss: 0.1144 | Acc: 0.9080 | Prec: 0.9081 | Rec: 0.9080 | F1: 0.9080

VALIDATION METRICS:
Loss: 0.1725 | Acc: 0.8892 | Prec: 0.9445 | Rec: 0.8892 | F1: 0.9046

Confusion Matrix:
[[5078,   86]]  <- Real
[[5437, 39264]]  <- Fake
Average Confidence: 0.4962
🎉 NEW BEST MODEL SAVED! Val Acc: 0.8892

EPOCH 2/5


Epoch 2 [Train]:   0%|          | 0/3173 [00:00<?, ?it/s]

...

Epoch 3 [Train]:   0%|          | 0/3173 [00:00<?, ?it/s]

Epoch 3 [Val]:   0%|          | 0/6234 [00:00<?, ?it/s]


TRAIN METRICS:
Loss: 0.0304 | Acc: 0.9790 | Prec: 0.9790 | Rec: 0.9790 | F1: 0.9790

VALIDATION METRICS:
Loss: 0.0982 | Acc: 0.9375 | Prec: 0.9585 | Rec: 0.9375 | F1: 0.9432

Confusion Matrix:
[[5027,  137]]  <- Real
[[2980, 41721]]  <- Fake
Average Confidence: 0.4990

EPOCH 4/5


Epoch 4 [Train]:   0%|          | 0/3173 [00:00<?, ?it/s]

Epoch 4 [Val]:   0%|          | 0/6234 [00:00<?, ?it/s]

In [ ]:
import torch

# Load the checkpoint
checkpoint = torch.load('best_improved_antispoofmodel.pth', map_location=device)

# Load model state dict
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()  # Set to evaluation mode

In [ ]:
# Reinitialize the trainer (optional, if you want metrics/logging)
trainer = ImprovedTrainer(model=model, device=device)

# Evaluate on the 30% subset
val_loss, val_acc, val_prec, val_rec, val_f1, cm, confidences = trainer.validate_epoch(
    val_loader=eval_loader_test, 
    epoch=0  # Epoch number doesn't matter for evaluation
)

# Print results
print(f"\nTEST RESULTS (30% EVAL SET):")
print(f"Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")
print(f"Precision: {val_prec:.4f} | Recall: {val_rec:.4f} | F1: {val_f1:.4f}")
print("\nConfusion Matrix:")
print(f"[[{cm[0,0]:4d}, {cm[0,1]:4d}]]  <- Real (bonafide)")
print(f"[[{cm[1,0]:4d}, {cm[1,1]:4d}]]  <- Fake (spoof)")

In [ ]:
from prettytable import PrettyTable

def model_size_table(model):
    table = PrettyTable(["Layer", "Params", "Size (MB)"])
    total_params = 0
    total_size = 0
    
    for name, param in model.named_parameters():
        params = param.numel()
        size = params * param.element_size() / (1024 ** 2)  # MB
        table.add_row([name, f"{params:,}", f"{size:.4f}"])
        total_params += params
        total_size += size
    
    print(table)
    print(f"\nTotal: {total_params:,} parameters, ~{total_size:.2f} MB")

model_size_table(model)

In [ ]:
torch.save(model.state_dict(), '/kaggle/working/antispoof_model.pth')

In [ ]:
# Initialize a fresh model instance
kmodel = AntiSpoofModel(
     wav2vec_model_path=wav2vec_model_path + "/wav2vec2-base",
    use_transformer=True,
    num_classes=2
).to('cuda' if torch.cuda.is_available() else 'cpu')

# Load the saved weights
kmodel.load_state_dict(torch.load('/kaggle/working/antispoof_model.pth'))
kmodel.eval()  # Set to evaluation mode


In [ ]:
def preprocess_audio(path, sample_rate=16000, max_len=48000):
    waveform, sr = torchaudio.load(path)
    if sr != sample_rate:
        waveform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=sample_rate)(waveform)
    waveform = waveform.mean(dim=0)  # Convert to mono
    if waveform.shape[-1] < max_len:
        pad = max_len - waveform.shape[-1]
        waveform = F.pad(waveform, (0, pad))
    else:
        waveform = waveform[:max_len]
    return waveform.unsqueeze(0)  # [1, T]

def predict_single(model, audio_path, device='cuda'):
    model.eval()
    waveform = preprocess_audio(audio_path).to(device)  # [1, T]
    with torch.no_grad():
        output = model(waveform)
        probs = F.softmax(output['logits'], dim=1)
        pred = torch.argmax(probs, dim=1).item()
        confidence = output['confidence'].item()
    
    label = "Real (Bonafide)" if pred == 0 else "Fake (Spoof)"
    return label, confidence


for path in test_dataset:
    label, conf = predict_single(model, path)
    print(f"{path} => Prediction: {label}, Confidence: {conf:.2f}")
